In [75]:
import numpy as np
import pandas as pd
import matplotlib
import math
import requests
import datetime
import regex as re
from scipy import stats


### Functions for loading the datasets

In [76]:
PATH = "KPMG_VI_New_raw_data_update_final.xlsx"

def load_address():
    columns = [
        "Customer ID",
        "Address",
        "Postcode",
        "State",
        "Country",
        "Property Valuation"
    ]
    return pd.read_excel(PATH, sheet_name="CustomerAddress", header = 1, names = columns)

def load_demograph():
    columns = [
        "Customer ID",
        "First Name",
        "Last Name",
        "Gender",
        '3YR Bike Related Purchases',
        "Date of Birth",
        "Job Title",
        "Job Industry",
        "Wealth Segament",
        "Deceased",
        "Default",
        "Owns Car",
        "Tenure"
    ]
    df = pd.read_excel(PATH, sheet_name="CustomerDemographic", header = 2, names = columns)
    df.drop('Default', axis=1, inplace=True)
    return df

def load_new_cust_list():
    columns = [
        "First Name",
        "Last Name",
        "Gender",
        "3YR Bike Related Purchases",
        "Date of Birth",
        "Job Title",
        "Job Industry",
        "Wealth Segament",
        "Deceased",
        "Owns Car",
        "Tenure",
        "Address",
        "Postcode",
        "State",
        "Country",
        "Property Valuation",
        "Rank",
        "Value"
        ] 
    df = pd.read_excel(PATH, sheet_name="NewCustomerList", header = 1, index_col= False)
    unnamed = df.loc[:,df.columns.str.match("Unnamed")].columns

    for index in range(len(unnamed)):
        df.drop(unnamed[index], axis = 1, inplace = True)

    old_names = df.columns
    df.rename(columns=dict(zip(old_names, columns)), inplace=True)
    
    return df
    
def load_transactions():
    columns = [
        "Transaction ID",
        "Product ID",
        "Customer ID",
        "Transaction Date",
        "Online Order",
        "Order Status",
        "Brand",
        "Product Line",
        "Product Class",
        "Product Size",
        "List Price",
        "Standard Cost",
        "Product First Sold Date"
    ] 
    df = pd.read_excel(PATH, sheet_name="Transactions", header = 1, names = columns)
    df['Transaction Date'] = pd.to_datetime(df['Transaction Date'], unit='s')
    return df

# Error Summary

### Customer Address

- State
    - The state values are inconsistent - the dataset contains VIC and Victoria, both of which are representing the same state. Same goes for NSW

### Customer Demographic

- Gender 
    - 1 value of Female is spelled incorrectly - Femal
    - 1 value of Male is just written as M

    In cases where we have to focus on a specific group - male or female, we would be missing out on 2 potential customers because it is very easy to overlook the fact that all values may not be spelled the correctly. This loss could easily be prevented by renaming the 2 fields

- Name
    - 125 Last Names are invalid 
    
    This shouldn't be too a problem since we can use the first name for marketing purposes instead of First Name and Last Name

- Date of Birth 
    - 3 people above the age of 83 (average life expectancy in Australia).
    - 87 Unknown values 

    It would be prudent to touch base with them and update their status as required.
    We would miss out on 87 potential customers campaigns which are sent out on their birthdays.

- Job Title
    - 506 unknown values

    The 506 Unknown values would make the analysis less accurate but it's unlikley that it would vary greatly from if the fields were filled out
- Job Industry
    - 656 unknown values

    The 656 Unknown values would make the analysis less accurate but it's unlikley that it would vary greatly from if the fields were filled out

- Tenure 
    - 87 unknown values. 

### Transactions

- Product ID
    - The product ID is not Unique. Brand, Product Line, Product Class, Product Size, List Price, and Standard Cost should be consistent across all the rows with the same product ID. 

    Either the product ID is not deterministic, or we need some more information to choose the correct product.


# General Observations:

Any sheet that contains 'Address' also contains - Postcode, State, and Country

Name is always 2 part - First Name, Last Name


## CustomerAddress

### Error Summary
- State
    - The state values are inconsistent - the dataset contains VIC and Victoria, both of which are representing the same state. Same goes for NSW


In [77]:
addresses = load_address()
addresses.head()

,Customer ID,Address,Postcode,State,Country,Property Valuation
0,1,060 Morning Avenue,2016,New South Wales,Australia,10
1,2,6 Meadow Vale Court,2153,New South Wales,Australia,10
2,4,0 Holy Cross Court,4211,QLD,Australia,9
3,5,17979 Del Mar Point,2448,New South Wales,Australia,4
4,6,9 Oakridge Court,3216,VIC,Australia,9


In [78]:
addresses.isnull().sum()

Customer ID           0
Address               0
Postcode              0
State                 0
Country               0
Property Valuation    0
dtype: int64

#### Datatype of the columns

In [79]:
addresses.dtypes

Customer ID            int64
Address               object
Postcode               int64
State                 object
Country               object
Property Valuation     int64
dtype: object

#### Checking Customer ID for invalid values, null, and duplicates 

In [80]:
def ID_check(df, field):
    CID_dupe = df[field].duplicated().sum()
    CID_null = df[field].isnull().sum()
    CID_invaid = (df[field] < 0).sum()
    
    print("ID - Duplicates:", CID_dupe, "| Null:", CID_null, "| Invalid:", CID_invaid)

ID_check(addresses, 'Customer ID')



ID - Duplicates: 0 | Null: 0 | Invalid: 0


All Customer IDs are valid

#### Fixing the State Values:
State values are inconsistent - Sometimes the value is VIC and sometimes it uses Victoria, both of which are referring to the same state. Same with NSW

In [81]:
def fix_state(df):
    state_dict = {
        'NSW' : 'New South Wales',
        'QLD' : 'Queensland',
        'SA' : 'South Australia',
        'TAS' : 'Tasmania',
        'VIC' : 'Victoria',
        'WA' : 'Western Australia',
        'ACT' : 'Australian Capital Territory',
        'NT' : 'Northern Territory'
    }
    for key, val in state_dict.items():
        df.replace(to_replace = val, value = key, inplace = True)

fix_state(addresses)


#### Address Validation

Checking for:
- all addresses are in Australia
- postal codes are within the range (data taken from Wikipedia)
- address has no invalid characters
- all states are non empty

In [82]:
def address_check(df):
    country_check = list()
    address_check = list()
    post_code_check = list()
    state_check = list()
    regex = re.compile('[@_!#$%^&*()<>?/\|}{~:]')

    for index in range(len(df)):
        country = df.loc[index, 'Country']
        address = df.loc[index, 'Address'] 
        state = df.loc[index, 'State'] 
        post_code = df.loc[index, 'Postcode'] 

        #Postal code range according to Wikipedia
        NSW_range = 1000 <= post_code <= 1999 or 2000 <= post_code <= 2599 or 2619 <= post_code <= 2899 or 2921 <= post_code <= 2999
        QLD_range = 4000 <= post_code <= 4999 or 9000 <= post_code <= 9999
        VIC_range = 3000 <= post_code <= 3999 or 8000 <= post_code <= 8999
        ACT_range = 200 <= post_code <= 299 or 2600 <= post_code <= 2618 or 2900 <= post_code <= 2920
        SA_range = 5000 <= post_code <= 5999
        WA_range = 6000 <= post_code <= 6797 or 6800 <= post_code <= 6999
        TAS_range = 7000 <= post_code <= 7999
        NT_range = 800 <= post_code <= 999

        code_ranges = {
            'NSW' : NSW_range,
            'QLD' : QLD_range, 
            'VIC': VIC_range, 
            'ACT' : ACT_range, 
            'SA' : SA_range,
            'WA' : WA_range,
            'TAS' : TAS_range,
            'NT' : NT_range
        }
        
        country_check.append(country != 'Australia')
        address_check.append(regex.search(address) != None)
        state_check.append(state not in code_ranges.keys())
        post_code_check.append(not code_ranges[state])
    
    print("Country:", sum(country_check), "Address:", sum(address_check), "State", sum(state_check), "Postcode:", sum(post_code_check))


address_check(addresses)




Country: 0 Address: 0 State 0 Postcode: 0


There are no issues with the addresses 

### Bonus Analysis - finding the best investment properties

This analysis creates a list of properties that are significnatly cheaper compared to the other properties in the neighbourhood.
These properties have potential to increase the value to the neighbourhood standard by doing renovations.

In [83]:
def find_investment(df, threshold = 20):
    postcode = addresses['Postcode'].value_counts()
    potential_postcodes = postcode.loc[postcode > threshold]
    potential_investments = dict()

    for code in potential_postcodes:
        valuations = df.loc[ df['Postcode'] == code]['Property Valuation']
        z = np.abs(stats.zscore(valuations))
        # We are choosing values that are signifigantly cheaper compared to the other properties in the neighbourhood 
        # To do this, choose the property with the min value in the neighbourhood and if it's an outlier too, it's a potential investment property
        outliers = z
        min_val = df.loc[df['Postcode'] == code].min()
        for key, outlier in outliers.items():
            if z.max() > 2.5:
                s = 1
        index_investment = 4



## Customer Demographic

### Error Summary:
- Gender 
    - 1 value of Female is spelled incorrectly - Femal
    - 1 value of Male is just written as M
- Name
    - 125 Last Names are invalid - This shouldn't be too a problem since we can use just the first name for marketing purposes
- Date of Birth 
    - 3 people above the age of 83 (average life expectancy in Australia). It would be prudent to touch base with them and update their status as required
    - 87 Unknown values 
- Job Title
    - 506 unknown values
- Job Industry
    - 656 unknown values
- Tenure 
    - 87 unknown values. The known values are good


The unknown values would make the data analysis less accurate, however, it is prudent to keep the rows in the table since the other fields in the row are valueable to analyse the sheet

In [84]:
demograph = load_demograph()
demograph.head()


/var/folders/n1/92hdfxcj7qscj2fkq7qqrt800000gn/T/ipykernel_2556/2104157337.py:30: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype={value.dtype})
  df = pd.read_excel(PATH, sheet_name="CustomerDemographic", header = 2, names = columns)


,Customer ID,First Name,Last Name,Gender,3YR Bike Related Purchases,Date of Birth,Job Title,Job Industry,Wealth Segament,Deceased,Owns Car,Tenure
0,2,Eli,Bockman,Male,81,1980-12-16,Administrative Officer,Financial Services,Mass Customer,N,Yes,16.0
1,3,Arlin,Dearle,Male,61,1954-01-20,Recruiting Manager,Property,Mass Customer,N,Yes,15.0
2,4,Talbot,NaN,Male,33,1961-10-03,NaN,IT,Mass Customer,N,No,7.0
3,5,Sheila-kathryn,Calton,Female,56,1977-05-13,Senior Editor,NaN,Affluent Customer,N,Yes,8.0
4,6,Curr,Duckhouse,Male,35,1966-09-16,NaN,Retail,High Net Worth,N,Yes,13.0


In [85]:
demograph.isnull().sum()

Customer ID                     0
First Name                      0
Last Name                     125
Gender                          0
3YR Bike Related Purchases      0
Date of Birth                  87
Job Title                     506
Job Industry                  656
Wealth Segament                 0
Deceased                        0
Owns Car                        0
Tenure                         87
dtype: int64

#### Customer ID

In [86]:
ID_check(demograph, 'Customer ID')

ID - Duplicates: 0 | Null: 0 | Invalid: 0


All Customer IDs are valid

#### Name Check
Since we have First and Last name in all the sheets which contain name, we can have a single function to verify if the name is valid

In [87]:
def name_check(df):
    regex = re.compile('[@_!#$%^&*()<>?/\|}{~:]')
    ret_dict = {'First': 0, 'Last': 0,'Both': 0}
    for i in range(len(df)):
        first = df.loc[i, 'First Name']
        last = df.loc[i, 'Last Name']
        
        first_valid = isinstance(first, str) and first.strip() and regex.search(first) == None
        last_valid = isinstance(last, str) and last.strip() and regex.search(last) == None

        if not (first_valid or last_valid):
            ret_dict['Both'] = ret_dict.get('Both') + 1
        elif not first_valid:
            ret_dict['First'] = ret_dict.get('First') + 1
        elif not last_valid:
            ret_dict['Last'] = ret_dict.get('Last') + 1
    print(ret_dict)

name_check(demograph)
        

{'First': 0, 'Last': 125, 'Both': 0}


#### Date of Birth Check
The average life expectancy in Australia is 82.9. The format is assumed to be 

In [88]:
demograph['Deceased'].value_counts()

N    3997
Y       2
Name: Deceased, dtype: int64

In [89]:
def DOB_check(df, max_age, min_age):
    curr_year = datetime.date.today().year
    overage_year = curr_year - max_age
    underage_year = curr_year - min_age
    overage_list = list()
    underage_list = list()
    deceased_list = list()
    unknown = demograph['Date of Birth'].isnull().sum()

    for index in range(len(df)):
        overage = df.loc[index, 'Date of Birth'].year < overage_year
        underage = df.loc[index, 'Date of Birth'].year > underage_year

        if df.loc[index, 'Deceased'] == 'N':
            overage_list.append(overage)
            underage_list.append(underage)
        else:
            deceased_list.append(True)

    print(f"Over {max_age}:", sum(overage_list), f"| Under {min_age}:", sum(underage_list), "| Deceased:", sum(deceased_list), "| Unknown:", unknown)

DOB_check(demograph, 83, 15)

Over 83: 3 | Under 15: 0 | Deceased: 2 | Unknown: 87


There are 3 people over the age of 83 in the Customer Demograph sheet that are not deceased.

The Date of Birt of 87 people is unknown

In [90]:
demograph['Gender'].value_counts()

Female    2037
Male      1872
U           88
Femal        1
M            1
Name: Gender, dtype: int64

In [91]:
demograph['Job Title'].value_counts()

Business Systems Development Analyst    45
Tax Accountant                          44
Social Worker                           44
Internal Auditor                        42
Legal Assistant                         41
                                        ..
Database Administrator II                4
Health Coach III                         3
Health Coach I                           3
Research Assistant III                   3
Developer I                              1
Name: Job Title, Length: 195, dtype: int64

In [92]:
demograph['Job Industry'].value_counts()

Manufacturing         799
Financial Services    774
Health                601
Retail                358
Property              267
IT                    223
Entertainment         136
Argiculture           113
Telecommunications     72
Name: Job Industry, dtype: int64

In [93]:
demograph['Wealth Segament'].value_counts()

Mass Customer        1999
High Net Worth       1021
Affluent Customer     979
Name: Wealth Segament, dtype: int64

In [94]:
demograph['Owns Car'].value_counts()

Yes    2023
No     1976
Name: Owns Car, dtype: int64

In [95]:
def tenure_check(df):
    tenure_list = list()
    for index in range(len(df)):
        tenure = df.loc[index, 'Tenure']
        tenure_check = not (isinstance(tenure, float) and 0 < tenure < 65 and not math.isnan(tenure))
        tenure_list.append(tenure_check)
    print("Invalid:", sum(tenure_list))

tenure_check(demograph)

Invalid: 87


## Transactions

### Error Summary
- Product ID
    - The product ID is not Unique. Brand, Product Line, Product Class, Product Size, List Price, and Standard Cost should be consistent across all the rows with the same product ID. The
- Product First Sold Date
    - This value is incomprehensible without more context. Regardless, this value does not seem important and therefore can be left out of the analysis

In [96]:
transactions = load_transactions()
transactions.head()

,Transaction ID,Product ID,Customer ID,Transaction Date,Online Order,Order Status,Brand,Product Line,Product Class,Product Size,List Price,Standard Cost,Product First Sold Date
0,1,2,2950,2017-02-25,0.0,Approved,Solex,Standard,medium,medium,71.49,53.62,41245.0
1,2,3,3120,2017-05-21,1.0,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92,41701.0
2,3,37,402,2017-10-16,0.0,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82,36361.0
3,4,88,3135,2017-08-31,0.0,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.10,36145.0
4,5,78,787,2017-10-01,1.0,Approved,Giant Bicycles,Standard,medium,large,1765.30,709.48,42226.0


In [98]:
transactions.isnull().sum()


Transaction ID               0
Product ID                   0
Customer ID                  0
Transaction Date             0
Online Order               360
Order Status                 0
Brand                      197
Product Line               197
Product Class              197
Product Size               197
List Price                   0
Standard Cost              197
Product First Sold Date    197
dtype: int64

Null values found 
- 360 Online Order Values - This shouldn't make much differnece since we have 20000 rows to analyse. Therefore, we can easily ignore the status or divide them according to the ratio of the rest of the transactions
- 197 Brand values, Product Line, Product Class, Product Size, Stamdard Cost values are unknown - this could easily be fixed using Product ID.


#### Transaction ID

In [99]:
ID_check(transactions, 'Transaction ID')

ID - Duplicates: 0 | Null: 0 | Invalid: 0


Transaction IDs are valid
#### Product ID

In [100]:
ID_check(transactions, 'Product ID')

ID - Duplicates: 19899 | Null: 0 | Invalid: 0


All Product IDs ara valid - Duplicates are acceptable 

#### Product Validation
Checking if Brand, Product line, Product Class, Product Size, are consistent across the product IDs

In [101]:
def consistency_check_PID(df):
    check_dict = dict()
    bool_list = list()
    brand_list = list()
    p_line_list = list()
    p_class_list = list()
    p_size_list = list()
    l_price_list = list()
    std_cost_list = list()

    for index in range(len(df)):
        PID = df.loc[index, 'Product ID']
        brand = df.loc[index, 'Brand']
        p_line = df.loc[index, 'Product Line']
        p_class = df.loc[index, 'Product Class']
        p_size = df.loc[index, 'Product Size']
        l_price = df.loc[index, 'List Price']
        std_cost = df.loc[index, 'Standard Cost']

        check_dict[PID] = check_dict.get(PID, (brand, p_line, p_class, p_size, l_price, std_cost)) 
        temp_tuple = check_dict[PID]

        brand_check = temp_tuple[0] != brand
        p_line_check = temp_tuple[1] != p_line
        p_class_check = temp_tuple[2] != p_class
        p_size_check = temp_tuple[3] != p_size
        l_price_check = temp_tuple[4] != l_price
        std_cost_check = temp_tuple[5] != std_cost

        brand_list.append(brand_check)
        p_line_list.append(p_line_check)
        p_class_list.append(p_class_check)
        p_size_list.append(p_size_check)
        l_price_list.append(l_price_check)
        std_cost_list.append(std_cost_check)

    print("Brands: ", sum(brand_list), "| Product Line:", sum(p_line_list), "| Product Class:", sum(p_class_list),"| Product Size:", sum(p_size_list),"| List Price:", sum(l_price_list), "| Standard Cost:", sum(std_cost_list))

consistency_check_PID(transactions)


Brands:  5878 | Product Line: 3572 | Product Class: 3275 | Product Size: 3007 | List Price: 6747 | Standard Cost: 6747


This shows that Proudct ID is not unique and needs another variable to determine the correct product since 1000s of rows that share the same product ID have a different value for the other attributes

In [102]:
transactions[transactions['Product ID'] == 0]

,Transaction ID,Product ID,Customer ID,Transaction Date,Online Order,Order Status,Brand,Product Line,Product Class,Product Size,List Price,Standard Cost,Product First Sold Date
34,35,0,2171,2017-08-20,0.0,Approved,Norco Bicycles,Road,medium,medium,544.05,376.84,38647.0
39,40,0,2448,2017-11-28,1.0,Approved,Norco Bicycles,Road,medium,medium,544.05,376.84,38647.0
54,55,0,3140,2017-09-18,0.0,Approved,Norco Bicycles,Standard,low,medium,363.01,290.41,38482.0
60,61,0,1839,2017-02-24,0.0,Approved,OHM Cycles,Road,high,large,12.01,7.21,34244.0
63,64,0,2000,2017-07-08,0.0,Approved,Trek Bicycles,Standard,medium,medium,499.53,388.72,39031.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19921,19922,0,1869,2017-01-06,1.0,Approved,Norco Bicycles,Road,medium,medium,543.39,407.54,37698.0
19941,19942,0,966,2017-07-30,1.0,Approved,OHM Cycles,Standard,low,medium,71.16,56.93,38647.0
19967,19968,0,2751,2017-04-06,0.0,Approved,WeareA2B,Standard,medium,medium,60.34,45.26,34165.0
19987,19988,0,13,2017-04-05,1.0,Approved,Norco Bicycles,Road,medium,medium,544.05,376.84,42105.0


In [103]:
transactions['Online Order'].value_counts()

1.0    9829
0.0    9811
Name: Online Order, dtype: int64

Assumption - 1 is online and 0 is otherwise

In [104]:
transactions['Order Status'].value_counts()

Approved     19821
Cancelled      179
Name: Order Status, dtype: int64

In [105]:
transactions['Product Line'].value_counts()

Standard    14176
Road         3970
Touring      1234
Mountain      423
Name: Product Line, dtype: int64

In [106]:
transactions['Product Class'].value_counts()

medium    13826
high       3013
low        2964
Name: Product Class, dtype: int64

In [107]:
transactions['Product Size'].value_counts()

medium    12990
large      3976
small      2837
Name: Product Size, dtype: int64

## Cross Checks

#### Error Summary
- 

### Transactions and Customer Demographic
Check whether all Customer IDs from transactions are present in Cutomer Demographic 


In [108]:
def cross_CID_check(df1, df2):
    bool_list = list()
    for index in range(len(df1)):
        CID = df1.loc[index, 'Customer ID']
        is_in = CID in df2['Customer ID']  
        bool_list.append(not is_in)
    print(sum(bool_list))

cross_CID_check(transactions, demograph)

3


3 Customers from Transactions are not present in the Demograph database

#### Customer Address and Demographics
Checking whether all Customer IDs from Customer IDs are present in Demographics

In [109]:
cross_CID_check(demograph, addresses)

2


2 Customers from Customer Demograph do not have their address in the Address database